In [1]:
# Tyler Palmer
# Student ID: 801058786
# Github: https://github.com/TPal49
# Homework 3

import time
import torch
import torchvision
from torch import nn
from matplotlib import pyplot as plt
import torchvision.datasets as datasets
from torchvision import transforms
import torch.optim as optim
import numpy as np
from torch.nn import functional as F



In [2]:
train = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
test = datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
!pip install d2l==1.0.0-beta0
!pip install ptflops
from d2l import torch as d2l
d2l.use_svg_display()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=dca367f36cec5229c252f05a0f71bdca7b5960026bb1525f37ce5cfb3e1d2473
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstal

In [4]:

def vgg_block(num_convs, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.LazyConv2d(out_channels, kernel_size=3, padding=1))
        layers.append(nn.LazyBatchNorm2d())
        layers.append(nn.ReLU())
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

class VGG(d2l.Classifier):
    def __init__(self, arch, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        conv_blks = []
        for (num_convs, out_channels) in arch:
            conv_blks.append(vgg_block(num_convs, out_channels))
        self.net = nn.Sequential(
            *conv_blks, nn.Flatten(),
            nn.LazyLinear(256), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(256), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)






In [5]:

transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data/',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False)




Files already downloaded and verified
Files already downloaded and verified


question 1 - vgg11

In [14]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 64), (2, 32))) # VGG 11
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.1, momentum = 0.9)

epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  75.49
Model accuracy on validation:  71.86
Model Loss:  0.9253934025764465


question 1 - vgg16

In [15]:
model2 = VGG(arch=((2, 16), (2, 32), (3, 64), (3, 128), (3, 128)), lr=0.1) # VGG 16
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.1)
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  75.484
Model accuracy on validation:  71.78
Model Loss:  1.0784943103790283


question 1  - vgg19

In [20]:
model = VGG(arch=((2, 16), (2, 32), (4, 64), (4, 128), (4, 128))) # VGG 19
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.1, momentum = 0.9)
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  57.01199999999999
Model accuracy on validation:  55.76
Model Loss:  1.1473625898361206


question 2

In [16]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

class GoogleNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
@d2l.add_to_class(GoogleNet)
def b2(self):
  return nn.Sequential(
      nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
      nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
  
@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(Inception(64, (96, 128), (16, 32), 32),
                         Inception(128, (128, 192), (32, 96), 64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
  
@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(Inception(192, (96, 208), (16, 48), 64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(Inception(256, (160, 320), (32, 128), 128),
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())
    
@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    super(GoogleNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(num_classes))
    self.net.apply(d2l.init_cnn)



In [17]:
model = GoogleNet(lr=0.1)
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.1, momentum = 0.9)
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  66.266
Model accuracy on validation:  63.349999999999994
Model Loss:  1.197005271911621


Question 3

In [18]:
class Residual(nn.Module):  
    """The Residual block of ResNet models."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

class ResNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
@d2l.add_to_class(ResNet)
def block(self, num_residuals, num_channels, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels))
    return nn.Sequential(*blk)

@d2l.add_to_class(ResNet)
def __init__(self, arch, lr=0.1, num_classes=10):
    super(ResNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, b in enumerate(arch):
        self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
    self.net.add_module('last', nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)





In [19]:
class ResNet18(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr, num_classes)

model = ResNet18(lr=0.01)
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.9)
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  81.54400000000001
Model accuracy on validation:  72.83
Model Loss:  0.641405463218689


In [22]:
class ResNet26(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128),(2, 128),(2, 128), (2, 256), (2, 512)),
                       lr, num_classes)

model = ResNet26(lr=0.01)
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.9)
epochs = 2
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  60.3
Model accuracy on validation:  58.050000000000004
Model Loss:  1.1516716480255127


In [23]:
class ResNet32(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128),(2, 128),(2, 128),(2, 128),(2, 128), (1, 256), (1, 512)),
                       lr, num_classes)

model = ResNet32(lr=0.01)
lossFunct = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.9)
epochs = 2
for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossFunct(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss = loss.item()
            

print('Finished Training.')

total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

val_total_correct = 0
val_total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total_images += labels.size(0)
        val_total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
val_model_accuracy = val_total_correct/ val_total_images * 100
print("Model accuracy on train: ", model_accuracy)
print("Model accuracy on validation: ", val_model_accuracy)
print("Model Loss: ", running_loss)

Finished Training.
Model accuracy on train:  53.886
Model accuracy on validation:  52.470000000000006
Model Loss:  1.4039490222930908
